In [1]:
from web3 import Web3
from web3.middleware import geth_poa_middleware
import json

# IUiPoolDataProviderV3 contract address
DATA_PROVIDER_CONTRACT_ADDRESS = "0x3De59b6901e7Ad0A19621D49C5b52cC9a4977e52"

# IPoolAddressesProvider contract address
POOL_ADDRESSES_PROVIDER_CONTRACT_ADDRESS = "0x4dd5ab8Fb385F2e12aDe435ba7AFA812F1d364D0"

# User address for fetching user reserves data
USER_ADDRESS = "0x9A560C3ceBB3a8c0f28Efcc43f4D12e942F4eeD7"

# ABI files for IUiPoolDataProviderV3 and IPoolAddressesProvider contracts
# You can get the ABI files from Etherscan or compile the contracts using solc
with open("IUiPoolDataProviderV3_ABI.json") as f:
    DATA_PROVIDER_ABI = json.load(f)
with open("IPoolAddressesProvider_ABI.json") as f:
    POOL_ADDRESSES_PROVIDER_ABI = json.load(f)

w3 = Web3(Web3.HTTPProvider('https://goerli.infura.io/v3/6f900384657d4152b0627543450ce5e7'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

data_provider_contract = w3.eth.contract(
    address=DATA_PROVIDER_CONTRACT_ADDRESS,
    abi=DATA_PROVIDER_ABI
)

pool_addresses_provider_contract = w3.eth.contract(
    address=POOL_ADDRESSES_PROVIDER_CONTRACT_ADDRESS,
    abi=POOL_ADDRESSES_PROVIDER_ABI
)

# 1. Fetch GHO's reserve data
def get_reserves_data():
    reserves_data, base_currency_info = data_provider_contract.functions.getReservesData(
        pool_addresses_provider_contract.address).call()
    return reserves_data, base_currency_info

# 2. Fetch GHO's user reserve data
def get_user_reserves_data(user_address):
    user_reserves_data, user_borrowing_power = data_provider_contract.functions.getUserReservesData(
        pool_addresses_provider_contract.address, user_address).call()
    return user_reserves_data, user_borrowing_power

reserves_data, base_currency_info = get_reserves_data()
print("Reserves Data:", reserves_data)
print("Base Currency Info:", base_currency_info)

user_reserves_data, user_borrowing_power = get_user_reserves_data(USER_ADDRESS)
print("User Reserves Data:", user_reserves_data)
print("User Borrowing Power:", user_borrowing_power)


Reserves Data: [('0xE205181Eb3D7415f15377F79aA7769F846cE56DD', 'AAVE', 'AAVE', 18, 5000, 6500, 11000, 0, True, False, False, True, False, 1000000000000000000000000000, 1000000000000000000000000000, 0, 0, 90000000000000000000000000, 1683984480, '0xAC4D92980562Ac11Af46C6C7CEdD7C819C2028D0', '0x1721dDa383B02ec058Ee7B47596F61246eAD0069', '0xCB62E1d181179d1D86D3877e221D1EdE0bDD8841', '0x13Bdaf61b68b917114D4A356c2098D703D4C4aB7', 2380107695761425690824975, 0, 0, 0, 0, 30000000000, '0xFAa450873a4b162D32f545a85e678C13Ca8d4Ae9', 70000000000000000000000000, 3000000000000000000000000000, 0, 0, 90000000000000000000000000, 0, 450000000000000000000000000, False, False, 0, 0, 0, True, 0, 2, 0, 0, 0, 0, 0, 0, '0x0000000000000000000000000000000000000000', '', False), ('0xD77b79BE3e85351fF0cbe78f1B58cf8d1064047C', 'DAI', 'DAI', 18, 7500, 8000, 10500, 1000, True, True, True, True, False, 1000241252314635863066636284, 1001494533949369579862651760, 901268939241428697697776, 5743095913579504993593753, 62871